In [37]:
from mantid.simpleapi import *
from mantid.api import MatrixWorkspace
from mantid.api import AnalysisDataService as ADS
from reduction_workflow.command_interface import ReductionSingleton, Clear
from mantid.api import *
from mantid.kernel import *
from sansdata import *

## 1. Create `Workspace` from data and instrument definition

In [ ]:
sans_file = SansData("data/memb_BS_Q1_6_0Ang.mpa")
sans_file.plot_2d()

In [ ]:
from reduction_workflow.command_interface import *
nBins = 1

delta_L_over_L0 = 0.1
bin_lower = sans_file.L0 * (1.0 - delta_L_over_L0/2)
bin_upper = sans_file.L0 * (1.0 + delta_L_over_L0/2)

detectors = 1024 * 1024

# Define single wavelength bin
x = np.tile([bin_lower,bin_upper],detectors)

print(x)
# Counts
y = sans_file.raw_intensity
# The workspace will be named "dataWS1", error values will be zero.
ws = CreateWorkspace(OutputWorkspace = "raw_2D_counts",UnitX="Wavelength",DataX=x, DataY=y, NSpec=detectors)
mon = LoadInstrument(ws, FileName="RIDSANS_Definition.xml", RewriteSpectraMap=True)


## 2. Verify basic Workspace properties

In [ ]:
# Print some basic instrument and workspace information
inst = ws.getInstrument()
di2 = ws.detectorInfo()
ci2 = ws.componentInfo()
print("Workspace {0} has instrument: {1}".format(ws.name(), inst.getName()))
print("Instrument {0} has {1} components, including {2} monitors and {3} detectors".format(inst.getName(), ci2.size(), len(mon), di2.size()))

# Get the sample position
instrument = ws.getInstrument()
sample = instrument.getSample()
sample_position = sample.getPos()
print("Sample Position:", sample_position)

blocksize = ws.blocksize()
print("Detector spectrum blocksize:", blocksize)


The following code probes the histograms of a cross section of the detectors, looking for detectors with a particularly high count. It also verifies that the bin (X-axis) is correct.

In [ ]:
print("number of histograms = {0}".format(ws.getNumberHistograms()))
# ws.getSpectrum(3)
for i in range(0,ws.getNumberHistograms(),100):
    sum_counts = ws.readY(i)[0]

    # Display spectrum number against sum_counts
    if sum_counts >= 100.0:
        print("{0} {1} {2}".format(ws.getSpectrum(i).getSpectrumNo(), sum_counts,ws.readX(i)))

In [ ]:
delta_Q = 1e-11
max_QXY = 1e-8
if ws.getRun().hasProperty("sample_detector_distance"):
    sample_detector_distance = ws.getRun().getProperty("sample_detector_distance").value
    print(sample_detector_distance)
Qxy(ws,MaxQxy=max_QXY, DeltaQ=delta_Q,OUTPUTWORKSPACE="new")

In [ ]:
reduced_ws = ADS.retrieve('new')
# dir(reduced_ws)
x = reduced_ws.dataX(14)
print(x.shape)
y = reduced_ws.dataY(21)
E = reduced_ws.dataE(21)
print(y.shape)
print(y.shape)

print(x,y,E)